# Week 2 Project

Once again, we’ll be using the [Women's Ecommerce Clothing Reviews Dataset from Kaggle](https://www.kaggle.com/datasets/nicapotato/womens-ecommerce-clothing-reviews), which Kaggle states is a 

> dataset revolving around the reviews written by customers. Its nine supportive features offer a great environment to parse out the text through its multiple dimensions. Because this is real commercial data, it has been anonymized, and references to the company in the review text and body have been replaced with “retailer”. 

The machine learning task will be sentiment analysis, classifying each review as having positive or negative sentiment.

## Task 1: Training and Evaluating Sentiment Analysis Models Using Metaflow

In this task, you'll use Metaflow to build two machine learning models for sentiment analysis: a baseline *"majority class"* classifier and your own custom model. You'll then train both models in parallel and experiment with different hyperparameters to optimize their performance. Finally, you'll use this notebook and the Metaflow Client API to analyze the results of your different models and hyperparameters. Here's what you'll need to do:

### Step 1: Build the Workflows
The first step in this task is to build the workflow(s) for sentiment analysis using the Metaflow framework. Start by creating a new flow in Metaflow and implementing the baseline *"majority class"* classifier. Then, build your own custom classifier using techniques you learned in Week 1, or any [helpful resources](https://outerbounds.com/docs/nlp-tutorial-L2/) you'd like. For your custom model, be sure to include steps for data preprocessing, model training, and evaluation.

### Step 2: Train Both Models in Parallel
Once you've built your models, the next step is to train both models in parallel using the Metaflow framework. Use Metaflow to run both training jobs in parallel steps. If you get stuck, you may want to review the [FlowSpec branching documentation](https://docs.metaflow.org/metaflow/basics#branch).

### Step 3: Experiment with Hyperparameters
After you've trained both models in parallel, the next step is to experiment with different hyperparameters to optimize their performance. Try different values for hyperparameters such as learning rate, batch size, and number of epochs, and record the results for each combination of hyperparameters as Data Artifacts in Metaflow.

### Step 4: Analyze the Results
Finally, use this notebook and the Metaflow Client API to analyze the results of your different models and hyperparameters. Create visualizations to compare the performance of the two models and identify the best hyperparameters for each one.

By completing this task, you'll gain experience working with the Metaflow framework and learn how to build and optimize machine learning workflows for sentiment analysis.

In [1]:
from collections import Counter
import pandas as pd
import numpy as np 
from termcolor import colored
import matplotlib.pyplot as plt
import seaborn as sns
import string

# You can style your plots here, but it is not part of the project.
YELLOW = '#FFBC00'
GREEN = '#37795D'
PURPLE = '#5460C0'
BACKGROUND = '#F4EBE6'
colors = [GREEN, PURPLE]
custom_params = {
    'axes.spines.right': False, 'axes.spines.top': False,
    'axes.facecolor':BACKGROUND, 'figure.facecolor': BACKGROUND, 
    'figure.figsize':(8, 8)
}
sns_palette = sns.color_palette(colors, len(colors))
sns.set_theme(style='ticks', rc=custom_params)

In [3]:
# done: load the data. 
filepath = "/home/workspace/workspaces/full-stack-ml-metaflow-corise-week-1"

df = pd.read_csv(filepath+"/data/Womens Clothing E-Commerce Reviews.csv", index_col=0)
df.head(2) 

,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses


In [4]:
# @futureme: update labeling fuction logic from week 1

def labeling_function(row):
    """
    A function to derive labels from the user's review data.
    This could use many variables, or just one. 
    In supervised learning scenarios, this is a very important part of determining what the machine learns!
   
    A subset of variables in the e-commerce fashion review dataset to consider for labels you could use in ML tasks include:
        # rating: Positive Ordinal Integer variable for the product score granted by the customer from 1 Worst, to 5 Best.
        # recommended_ind: Binary variable stating where the customer recommends the product where 1 is recommended, 0 is not recommended.
        # positive_feedback_count: Positive Integer documenting the number of other customers who found this review positive.

    In this case, we are doing sentiment analysis. 
    To keep things simple, we use the rating only, and return a binary positive or negative sentiment score based on an arbitrarty cutoff. 
    """
    # Done (v0): Add your logic for the labelling function here
    # It is up to you on what value to choose as the cut off point for the postive class
    # A good value to start would be 4
    # This function should return either a 0 or 1 depending on the rating of a particular row
    
    # Try catching some invalid values
    # if row.rating.dtype not in ('int64', 'float64'):
    #     return 0
    
    if row.rating >= 4:
        return 1
    else:
        return 0

In [6]:
# transformations
df.columns = ["_".join(name.lower().strip().split()) for name in df.columns]
df = df[~df.review_text.isna()]
df['review'] = df['review_text'].astype('str')
_has_review_df = df[df['review_text'] != 'nan']
reviews = _has_review_df['review_text']
labels = _has_review_df.apply(labeling_function, axis=1)
df = pd.DataFrame({'label': labels, **_has_review_df})

df.head()

/tmp/ipykernel_20176/4253732853.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'] = df['review_text'].astype('str')


,label,clothing_id,age,title,review_text,rating,recommended_ind,positive_feedback_count,division_name,department_name,class_name,review
0,1,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates,Absolutely wonderful - silky and sexy and comf...
1,1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses,Love this dress! it's sooo pretty. i happene...
2,0,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses,I had such high hopes for this dress and reall...
3,1,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants,"I love, love, love this jumpsuit. it's fun, fl..."
4,1,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses,This shirt is very flattering to all due to th...


In [7]:
# Testing the code to be deployed in FlowSpec

# load libraries 
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
# from sklearn.preprocessing import MinMaxScalar
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.svm import SVC
import sklearn.metrics as metric
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.dummy import DummyClassifier

"Split the data into training (fit), validation (trial run), test (final exam) sets"
SEED = 89
split_size = 0.2
_test_ratio = 0.4 # Default to 40% of the non-training data to be test set (train:val:test == 0.8:0.12:0.08)

def train_validation_test_split (
    X, y, train_ratio: float, validation_ratio: float, test_ratio: float
):
    # Split up dataset into train and test, of which we split up the test.
    X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=(1- train_ratio), random_state=SEED
    )

    # Split up test into two (validation and test).
    X_val, X_test, y_val, y_test = train_test_split(
    X_test, y_test, test_size=(test_ratio / (test_ratio + validation_ratio)), random_state=SEED,
    )

    # Return the splits
    return X_train, X_val, X_test, y_train, y_val, y_test

# features (X), label (y)
X = df.iloc[:, ~df.columns.isin(['label'])]
y = df[['label']]


X_train, X_val, X_test, y_train, y_val, y_test = train_validation_test_split(
    X, y, 1.0-split_size, (1.0-_test_ratio)*(1.0-split_size), _test_ratio*(1.0-split_size)
)

print(f'num of rows in train set: {X_train.shape[0]}')
print(f'num of rows in validation set: {X_val.shape[0]}')
print(f'num of rows in test set: {X_test.shape[0]}')

num of rows in train set: 18112
num of rows in validation set: 2717
num of rows in test set: 1812


In [8]:
y_val.head()

,label
2661,0
2091,0
20968,0
10710,1
16833,1


In [ ]:
# boilerplate from repo
# _df = pd.DataFrame({'review': reviews, 'label': labels})
# traindf, valdf = train_test_split(_df, test_size=0.2)

In [9]:
# TODO: build the dummy classfier baseline model.

#  Next step
"Numerical features"
_num_features_ls = X_train.select_dtypes(include=['int64', 'float64']).columns.values
"Categorical features"
_cat_features_ls = X_train.select_dtypes(include='category').columns.values

"Compute the baseline"
# Basic baseline can be as simple as predicting the most frequent class for a classification problem
# _pct_positive_sentiment = self.traindf.labels.sum() / self.traindf.labels.shape[0]

dummy_model = DummyClassifier(strategy="most_frequent")
dummy_model.fit(X_train, y_train)

model = dict()
model[0] = dummy_model

### Done: Fit and score a baseline model on the data, log the acc and rocauc as artifacts.
base_acc = dummy_model.score(X_train, y_train)
base_rocauc = 0.5 # AUC will be 0.5 for a dummy or random baseline model

dummy_y_predict = dummy_model.predict(X_train)
# base_acc_1 = accuracy_score(y_train, dummy_y_predict)
# base_rocauc_1 = roc_auc_score(y_train, dummy_y_predict)

print(f'Baseline model accuracy:  {base_acc}')
print(f'Baseline AUC: {base_rocauc}') 


Baseline model accuracy:  0.7715878975265018
Baseline AUC: 0.5


In [ ]:
# TODO: find the majority class in the labels. 🤔
# TODO: score the model on valdf with a 2D metric space: sklearn.metrics.accuracy_score, sklearn.metrics.roc_auc_score
    # Documentation on suggested model-scoring approach: https://scikit-learn.org/stable/modules/model_evaluation.html

In [10]:
import tensorflow as tf
from tensorflow.keras import layers, optimizers, regularizers
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.layers import Dense, LeakyReLU

class NbowModel():
    def __init__(self, vocab_sz):

        self.vocab_sz = vocab_sz

        # Instantiate the CountVectorizer
        self.cv = CountVectorizer(
            min_df=.0025, max_df = .5, stop_words='english', # reduce min freq to 0.25%, and max freq to 50%.
            strip_accents='ascii', max_features=self.vocab_sz
        )

        # Define the keras model
        # boilerplate from week-2 below
        # inputs = tf.keras.Input(shape=(self.vocab_sz,), 
        #                         name='input')
        # x = layers.Dropout(0.10)(inputs)
        # x = layers.Dense(
        #     15, activation="relu",
        #     kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4)
        # )(x)
        # predictions = layers.Dense(1, activation="sigmoid",)(x)
        # self.model = tf.keras.Model(inputs, predictions)
        # opt = optimizers.Adam(learning_rate=0.002)
        # self.model.compile(loss="binary_crossentropy", 
        #                    optimizer=opt, metrics=["accuracy"])

        # week-2 mucking around with keras model
        self.model = tf.keras.Sequential([
                    layers.Dense(10, input_shape=(self.vocab_sz,), activation=None, kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4)),
                    layers.LeakyReLU(alpha=0.1),
                    Dense(1, activation='sigmoid')
                ])
        self.model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
        

    def fit(self, X, y):
        print(X.shape)
        print(X[0])
        res = self.cv.fit_transform(X).toarray()
        self.model.fit(x=res, y=y, batch_size=32, 
                       epochs=10, validation_split=.2)
    
    def predict(self, X):
        print(X.shape)
        print(X[0])
        res = self.cv.transform(X).toarray()
        return self.model.predict(res)
    
    def eval_acc(self, X, labels, threshold=.5):
        return accuracy_score(labels, 
                              self.predict(X) > threshold)
    
    def eval_rocauc(self, X, labels):
        return roc_auc_score(labels,  self.predict(X))

In [11]:
# Experimentation phase: fit the custom model on data
model_test = NbowModel(vocab_sz=750)
model_test.fit(X=X_train["review"], y=y_train["label"])

(18112,)
Absolutely wonderful - silky and sexy and comfortable


2023-05-11 19:54:11.201375: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2023-05-11 19:54:11.220891: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2499995000 Hz


Epoch 1/10
453/453 [==============================] - 1s 1ms/step - loss: 0.4894 - accuracy: 0.7681 - val_loss: 0.3331 - val_accuracy: 0.8592
Epoch 2/10
453/453 [==============================] - 0s 1ms/step - loss: 0.3114 - accuracy: 0.8724 - val_loss: 0.3298 - val_accuracy: 0.8595
Epoch 3/10
453/453 [==============================] - 0s 1ms/step - loss: 0.2830 - accuracy: 0.8848 - val_loss: 0.3291 - val_accuracy: 0.8636
Epoch 4/10
453/453 [==============================] - 0s 1ms/step - loss: 0.2817 - accuracy: 0.8865 - val_loss: 0.3297 - val_accuracy: 0.8590
Epoch 5/10
453/453 [==============================] - 0s 1ms/step - loss: 0.2735 - accuracy: 0.8898 - val_loss: 0.3295 - val_accuracy: 0.8617
Epoch 6/10
453/453 [==============================] - 0s 1ms/step - loss: 0.2753 - accuracy: 0.8867 - val_loss: 0.3309 - val_accuracy: 0.8614
Epoch 7/10
453/453 [==============================] - 0s 1ms/step - loss: 0.2649 - accuracy: 0.8930 - val_loss: 0.3331 - val_accuracy: 0.8581
Epoch 

In [12]:
y_val['label'].isnull().values.any()

False

In [13]:
# Evaluate the exp. model perf (nlp-2.ipynb tutorial: https://outerbounds.com/docs/nlp-tutorial-L2/)

# valdf = pd.read_parquet('valid.parquet')
model_acc = model_test.eval_acc(
    X=X_train['review'], labels=y_train['label'])
model_rocauc = model_test.eval_rocauc(
    X=X_train['review'], labels=y_train['label'])

msg = 'Baseline Accuracy: {}\nBaseline AUC: {}'
print(msg.format(
    round(model_acc, 3), round(model_rocauc, 3)
))

# Having some issues with calling model.eval_acc() with X_val and y_val --> will debug later
# still don't understand why there is something being printed out ((18112,)
# Absolutely wonderful - silky and sexy and comfortable
# (18112,)
# Absolutely wonderful - silky and sexy and comfortable)

(18112,)
Absolutely wonderful - silky and sexy and comfortable
(18112,)
Absolutely wonderful - silky and sexy and comfortable
Baseline Accuracy: 0.899
Baseline AUC: 0.944


In [29]:
%%writefile model.py
# done: modify this custom model to your liking. Check out this tutorial for more on this class: https://outerbounds.com/docs/nlp-tutorial-L2/
import tensorflow as tf
from tensorflow.keras import layers, optimizers, regularizers
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.layers import Dense, LeakyReLU

class NbowModel():
    def __init__(self, vocab_sz):

        self.vocab_sz = vocab_sz

        # Instantiate the CountVectorizer
        self.cv = CountVectorizer(
            min_df=.0025, max_df = .5, stop_words='english', # reduce min freq to 0.25%, and max freq to 50%.
            strip_accents='ascii', max_features=self.vocab_sz
        )

        # Define the keras model
        # boilerplate from week-2 below
        inputs = tf.keras.Input(shape=(self.vocab_sz,), 
                                name='input')
        x = layers.Dropout(0.10)(inputs)
        x = layers.Dense(
            15, activation="relu",
            kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4)
        )(x)
        predictions = layers.Dense(1, activation="sigmoid",)(x)
        self.model = tf.keras.Model(inputs, predictions)
        opt = optimizers.Adam(learning_rate=0.002)
        self.model.compile(loss="binary_crossentropy", 
                           optimizer=opt, metrics=["accuracy"])

        # week-2 mucking around with keras model
        # self.model = tf.keras.Sequential([
        #             layers.Dense(10, input_shape=(self.vocab_sz,), activation=None, kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4)),
        #             layers.LeakyReLU(alpha=0.1),
        #             Dense(1, activation='sigmoid')
        #         ])
        
        # week-2 compile the model
        # self.model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
        

    def fit(self, X, y):
        print(X.shape)
        print(X[0])
        res = self.cv.fit_transform(X).toarray()
        self.model.fit(x=res, y=y, batch_size=32, 
                       epochs=10, validation_split=.2)
    
    def predict(self, X):
        print(X.shape)
        print(X[0])
        res = self.cv.transform(X).toarray()
        return self.model.predict(res)
    
    def eval_acc(self, X, labels, threshold=.5):
        return accuracy_score(labels, 
                              self.predict(X) > threshold)
    
    def eval_rocauc(self, X, labels):
        return roc_auc_score(labels,  self.predict(X))

    @property
    def model_dict(self): 
        return {'vectorizer':self.cv, 'model': self.model}

    @classmethod
    def from_dict(cls, model_dict):
        "Get Model from dictionary"
        nbow_model = cls(len(
            model_dict['vectorizer'].vocabulary_
        ))
        nbow_model.model = model_dict['model']
        nbow_model.cv = model_dict['vectorizer']
        return nbow_model


Overwriting model.py


In [67]:
# TODO: train the model on traindf.
# TODO: score the model on valdf with _the same_ 2D metric space you used in previous cell.
# TODO: test your model works by importing the model module in notebook cells, and trying to fit traindf and score predictions on the valdf data!

# # from model import NbowModel
# model = NbowModel(vocab_sz=750)
# model.fit(X=X_train["review"], y=y_train["label"])
# self.model_dict = model.model_dict  # save model
# self.next(self.join)

# model_acc = model.eval_acc(
#     valdf['review'], valdf['labels'])
# model_rocauc = model.eval_rocauc(
#     valdf['review'], valdf['labels'])

# msg = 'Baseline Accuracy: {}\nBaseline AUC: {}'
# print(msg.format(
#     round(model_acc, 3), round(model_rocauc, 3)
# ))

In [82]:
from metaflow import IncludeFile
data = IncludeFile('data', default='Womens Clothing E-Commerce Reviews.csv')

# _df = pd.read_csv(data)

# example:
# from metaflow import FlowSpec, IncludeFile
# import pandas as pd

# class MyFlow(FlowSpec):

#     # Include the CSV file in the flow
#     data_file = IncludeFile('my_data', default='my_data.csv')

#     def step1(self):
#         # Read the CSV file into a pandas DataFrame
#         df = pd.read_csv(self.data_file)
#         print(df.head())


In [17]:
# Let's try the baseline challenge

# split_size = Parameter('split-sz', default=0.2)
# data = IncludeFile('data', default='Womens Clothing E-Commerce Reviews.csv')
# kfold = Parameter('k', default=5)
# scoring = Parameter('scoring', default='accuracy')

labeling_function =  lambda x: 1 if x.rating >=4 else 0 # done: Define your labeling function here.
# def @step first step is to read the data from wherever and clean and save it as an artifact

import pandas as pd
import io 
from sklearn.model_selection import train_test_split

# load dataset packaged with the flow.
# this technique is convenient when working with small datasets that need to move to remove tasks.
_df_raw = df.copy() #for ipynb
# done: load the data in flowspec. 
# df = pd.read_csv(self.data)


# filter down to reviews and labels 
_df_raw.columns = ["_".join(name.lower().strip().split()) for name in _df_raw.columns]
_df_raw = _df_raw[~_df_raw.review_text.isna()]

_df_raw['review'] = _df_raw['review_text'].astype('str')
_has_review_df = _df_raw[_df_raw['review_text'] != 'nan']

reviews = _has_review_df['review_text']
labels = _has_review_df.apply(labeling_function, axis=1)
_df_clean = pd.DataFrame({'label': labels, **_has_review_df})

# split the data 80/20, or by using the flow's split-sz CLI argument
_df = pd.DataFrame({'review': reviews, 'label': labels})

# features (X), label (y)
X = _df.iloc[:, ~_df.columns.isin(['label'])]
y = _df[['label']]


X_train, X_val, X_test, y_train, y_val, y_test = train_validation_test_split(
    X, y, 1.0-split_size, (1.0-_test_ratio)*(1.0-split_size), _test_ratio*(1.0-split_size)
)

print(f'num of rows in train set: {X_train.shape[0]}')
print(f'num of rows in validation set: {X_val.shape[0]}')
print(f'num of rows in test set: {X_test.shape[0]}')

# self.traindf, self.valdf = train_test_split(_df, test_size=self.split_size)
# print(f'num of rows in train set: {self.traindf.shape[0]}')
# print(f'num of rows in validation set: {self.valdf.shape[0]}')

# self.next(self.baseline, self.model)

num of rows in train set: 18112
num of rows in validation set: 2717
num of rows in test set: 1812


In [ ]:
# how to create a picklable model object to serialize and store as artifact in order to deserialize later on
# from dataclasses import dataclass, asdict, fields
# self.result_serialized = asdict(ModelResult("Baseline", params, pathspec, acc, rocauc))

@dataclass
class ModelResult:
    "A custom struct for storing model evaluation results."
    name: None
    params: None
    pathspec: None
    acc: None
    rocauc: None

# def classFromArgs(className, argDict):
#     fieldSet = {f.name for f in fields(className) if f.init}
#     filteredArgDict = {k : v for k, v in argDict.items() if k in fieldSet}
#     return className(**filteredArgDict)

# data = Task('BaselineChallenge/87/baseline/428')['result_serialized'].data 
# print(classFromArgs(ModelResult, data))
# output:
# ModelResult(name='Baseline', params='Always predict 1', pathspec='BaselineChallenge/87/baseline/428', acc=0.7697063369397218, rocauc=0.5)

In [18]:
# basline step

"Compute the baseline"

from sklearn.metrics import accuracy_score, roc_auc_score

# these 3 arguments are needed to instantiate an object of the ModelResult class
# self._name = "baseline"
# params = "Always predict 1"
# pathspec = f"{current.flow_name}/{current.run_id}/{current.step_name}/{current.task_id}"

def getMajorityClass(df, col_label):
    return df[col_label].mode()[0]

predictions = getMajorityClass(_df, 'label') # done: predict the majority class
print(f'the majority label is {predictions}')

acc = model_test.eval_acc(
    X=_df['review'], 
    labels=pd.DataFrame({'label': [predictions] * _df.shape[0]})
    ) # done: return the accuracy_score of these predictions    

# ideally use the defined method with the model class for rocauc calc but Im getting an understandable error
# "ValueError: Only one class present in y_true. ROC AUC score is not defined in that case."
# rocauc = model_test.eval_rocauc(
#     X=_df['review'], 
#     labels=pd.DataFrame({'label': [predictions] * _df.shape[0]})
#  ) # done: return the roc_auc_score of these predictions

# People have bypassed this with below but using the y_val dataset always
rocauc = roc_auc_score(_df["label"],pd.DataFrame({'label': [predictions] * _df.shape[0]}))

print(f'Accuracy of majority class model is {round(acc*100, 2)}%, AUC of {rocauc}')

# self.result = ModelResult("Baseline", params, pathspec, acc, rocauc)
# self.next(self.aggregate)

the majority label is 1
(22641,)
Absolutely wonderful - silky and sexy and comfortable
Accuracy of majority class model is 79.77%, AUC of 0.5


In [29]:
# Testing the results from hyperperameter search

# def model(self):

# TODO: import your model if it is defined in another file.
from model import NbowModel
import pandas as pd
# self._name = "model"

# NOTE: If you followed the link above to find a custom model implementation, 
    # you will have noticed your model's vocab_sz hyperparameter.
    # Too big of vocab_sz causes an error. Can you explain why? 
# self.hyperparam_set = [{'vocab_sz': 100}, {'vocab_sz': 300}, {'vocab_sz': 500}]  
hyperparam_set = [{'vocab_sz': 100}, {'vocab_sz': 300}, {'vocab_sz': 500}]  
# pathspec = f"{current.flow_name}/{current.run_id}/{current.step_name}/{current.task_id}"
pathspec = f"/home/workspace/workspaces/full-stack-ml-metaflow-corise-week-2/data"

# self.results = []
results = []
for params in hyperparam_set: #self.hyperparam_set
    model = NbowModel(vocab_sz=150) # done: instantiate your custom model here!
    model.fit(X=X_train['review'], y=y_train['label'])
    try:
        acc = model.eval_acc(X_test['review'], y_test['label'])# TODO: evaluate your custom model in an equivalent way to accuracy_score.
        rocauc = model_rocauc = model.eval_rocauc(X_test['review'], y_test['label']) # TODO: evaluate your custom model in an equivalent way to roc_auc_score.
        results.append(ModelResult(f"NbowModel - vocab_sz: {params['vocab_sz']}", params, pathspec, acc, rocauc))
    except KeyError:
        results.append(ModelResult(f"NbowModel - vocab_sz: {params['vocab_sz']}", params, pathspec, None, None))
# self.next(self.aggregate)

(18112,)
Absolutely wonderful - silky and sexy and comfortable
Epoch 1/10
453/453 [==============================] - 1s 1ms/step - loss: 0.5270 - accuracy: 0.7329 - val_loss: 0.4032 - val_accuracy: 0.8109
Epoch 2/10
453/453 [==============================] - 0s 882us/step - loss: 0.4052 - accuracy: 0.8180 - val_loss: 0.4004 - val_accuracy: 0.8153
Epoch 3/10
453/453 [==============================] - 0s 879us/step - loss: 0.3928 - accuracy: 0.8221 - val_loss: 0.3998 - val_accuracy: 0.8156
Epoch 4/10
453/453 [==============================] - 0s 894us/step - loss: 0.3808 - accuracy: 0.8329 - val_loss: 0.3991 - val_accuracy: 0.8181
Epoch 5/10
453/453 [==============================] - 0s 912us/step - loss: 0.3801 - accuracy: 0.8337 - val_loss: 0.3976 - val_accuracy: 0.8222
Epoch 6/10
453/453 [==============================] - 0s 936us/step - loss: 0.3748 - accuracy: 0.8384 - val_loss: 0.3980 - val_accuracy: 0.8239
Epoch 7/10
453/453 [==============================] - 0s 916us/step - loss:

In [36]:
%%writefile baseline_challenge.py
# in progress: In this cell, write your BaselineChallenge flow in the baseline_challenge.py file.

from metaflow import FlowSpec, step, Flow, current, Parameter, IncludeFile, card, current
from metaflow.cards import Table, Markdown, Artifact, Image
import numpy as np 
from dataclasses import dataclass

labeling_function =  lambda x: 1 if x.rating >=4 else 0 # done: Define your labeling function here.

@dataclass
class ModelResult:
    "A custom struct for storing model evaluation results."
    name: None
    params: None
    pathspec: None
    acc: None
    rocauc: None

class BaselineChallenge(FlowSpec):

    split_size = Parameter('split-sz', default=0.2)
    _test_ratio = 0.4
    data = IncludeFile('data', default='Womens Clothing E-Commerce Reviews.csv')
    kfold = Parameter('k', default=5)
    scoring = Parameter('scoring', default='accuracy')

    @step
    def start(self):

        import pandas as pd
        import io 
        from sklearn.model_selection import train_test_split
        
        # load dataset packaged with the flow.
        # this technique is convenient when working with small datasets that need to move to remove tasks.
        # done: load the data in flowspec. 
        _df_raw = pd.read_csv(self.data)


        # filter down to reviews and labels 
        _df_raw.columns = ["_".join(name.lower().strip().split()) for name in _df_raw.columns]
        _df_raw = _df_raw[~_df_raw.review_text.isna()]

        _df_raw['review'] = _df_raw['review_text'].astype('str')
        _has_review_df = _df_raw[_df_raw['review_text'] != 'nan']

        reviews = _has_review_df['review_text']
        labels = _has_review_df.apply(labeling_function, axis=1)
        _df_clean = pd.DataFrame({'label': labels, **_has_review_df})

        # split the data 80/20, or by using the flow's split-sz CLI argument
        _df = pd.DataFrame({'review': reviews, 'label': labels})

        # features (X), label (y)
        X = _df.iloc[:, ~_df.columns.isin(['label'])]
        y = _df[['label']]

        "Split the data into training (fit), validation (trial run), test (final exam) sets"
        SEED = 89


        def train_validation_test_split (
            X, y, train_ratio: float, validation_ratio: float, test_ratio: float
        ):
            # Split up dataset into train and test, of which we split up the test.
            X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=(1- train_ratio), random_state=SEED
            )

            # Split up test into two (validation and test).
            X_val, X_test, y_val, y_test = train_test_split(
            X_test, y_test, test_size=(test_ratio / (test_ratio + validation_ratio)), random_state=SEED,
            )

            # Return the splits
            return X_train, X_val, X_test, y_train, y_val, y_test
            
        # split the data 80/20, or by using the flow's split-sz CLI argument
        self.X_train, self.X_val, self.X_test, self.y_train, self.y_val, self.y_test = train_validation_test_split(
            X, y, 1.0-split_size, (1.0-_test_ratio)*(1.0-split_size), _test_ratio*(1.0-split_size)
        )
        
        print(f'num of rows in train set: {self.X_train.shape[0]}')
        print(f'num of rows in validation set: {self.X_val.shape[0]}')

        self.next(self.baseline, self.model)

    @step
    def baseline(self):
        "Compute the baseline"

        from sklearn.metrics import accuracy_score, roc_auc_score
        import pandas as pd
        
        self._name = "baseline"
        params = "Always predict 1"
        pathspec = f"{current.flow_name}/{current.run_id}/{current.step_name}/{current.task_id}"
        def getMajorityClass(df, col_label):
            return df[col_label].mode()[0]

        predictions = getMajorityClass(_df, 'label') # done: predict the majority class
        # print(f'the majority label is {predictions}')

        acc = model_test.eval_acc(
            X=self.X_train['review'], 
            labels=pd.DataFrame({'label': [predictions] * self.X_train.shape[0]})
            ) # done: return the accuracy_score of these predictions    

        # ideally use the defined method with the model class for rocauc calc but Im getting an understandable error
        # "ValueError: Only one class present in y_true. ROC AUC score is not defined in that case."
        # rocauc = model_test.eval_rocauc(
        #     X=_df['review'], 
        #     labels=pd.DataFrame({'label': [predictions] * _df.shape[0]})
        #  ) # done: return the roc_auc_score of these predictions

        # People have bypassed this with below but using the y_val dataset always
        rocauc = roc_auc_score(self.X_train["label"],pd.DataFrame({'label': [predictions] * self.X_train.shape[0]}))
        # print(f'Accuracy of majority class model is {round(acc*100, 2)}%, AUC of {rocauc}')

        self.result = ModelResult("Baseline", params, pathspec, acc, rocauc)
        self.next(self.aggregate)

    @step
    def model(self):

        # done: import your model if it is defined in another file.
        from model import NbowModel
        import pandas as pd

        self._name = "model"
        # NOTE: If you followed the link above to find a custom model implementation, 
            # you will have noticed your model's vocab_sz hyperparameter.
            # Too big of vocab_sz causes an error. Can you explain why? 
        self.hyperparam_set = [{'vocab_sz': 100}, {'vocab_sz': 300}, {'vocab_sz': 500}]  
        pathspec = f"{current.flow_name}/{current.run_id}/{current.step_name}/{current.task_id}"

        self.results = []
        for params in self.hyperparam_set:
            model = NbowModel(vocab_sz=150) # done: instantiate your custom model here!
            model.fit(X=self.X_train['review'], y=self.y_train['label'])
            try:
                acc = model.eval_acc(self.X_test['review'], self.y_test['label'])# done: evaluate your custom model in an equivalent way to accuracy_score.
                rocauc = model.eval_rocauc(self.X_test['review'], self.y_test['label']) # done: evaluate your custom model in an equivalent way to roc_auc_score.
                self.results.append(ModelResult(f"NbowModel - vocab_sz: {params['vocab_sz']}", params, pathspec, acc, rocauc))
            except KeyError:
                self.results.append(ModelResult(f"NbowModel - vocab_sz: {params['vocab_sz']}", params, pathspec, None, None))

        self.next(self.aggregate)

    @card(type='corise')
    @step
    def aggregate(self, inputs):

        import seaborn as sns
        import matplotlib.pyplot as plt
        from matplotlib import rcParams 
        rcParams.update({'figure.autolayout': True})

        rows = []
        violin_plot_df = {'name': [], 'accuracy': []}
        for task in inputs:
            if task._name == "model": 
                for result in task.results:
                    print(result)
                    rows, violin_plot_df = self.add_one(rows, result, violin_plot_df)
            elif task._name == "baseline":
                print(task.result)
                rows, violin_plot_df = self.add_one(rows, task.result, violin_plot_df)
            else:
                raise ValueError("Unknown task._name type. Cannot parse results.")
            
        current.card.append(Markdown("# All models from this flow run"))

        
        current.card.append(
            Table(
                rows,
                headers=["Model name", "Params", "Task pathspec", "Accuracy", "ROCAUC"]
            )
        )
        
        fig, ax = plt.subplots(1,1)
        plt.xticks(rotation=40)
        sns.violinplot(data=violin_plot_df, x="name", y="accuracy", ax=ax)
        
        # TODO: Append the matplotlib fig to the card
        # Docs: https://docs.metaflow.org/metaflow/visualizing-results/easy-custom-reports-with-card-components#showing-plots
        current.card.append(Image.from_matplotlib(fig))
        self.next(self.end)

    @step
    def end(self):
        pass

if __name__ == '__main__':
    BaselineChallenge()

Overwriting baseline_challenge.py


## Task 2: Anticipating Failure in Your Machine Learning Project

In this task, you'll practice anticipating potential failure modes in a sentiment analysis classifier and develop strategies to mitigate them. Here's what you'll need to do:
### Step 1: Identify Potential Failure Modes

The first step in anticipating failure in your machine learning project is to identify potential failure modes. Start by brainstorming ways in which your project could fail from an engineering point of view. For example, your model could overfit to the training data or suffer from data bias.
### Step 2: Develop Strategies to Mitigate Failure Modes

Once you've identified potential failure modes, the next step is to develop strategies to mitigate them. Think about what measures you could take to fix the issue if it were to occur. For example, if your model is overfitting to the training data, you could try regularization techniques such as L1 or L2 regularization to reduce the complexity of the model.
### Step 3: Plan Ahead to Avoid Failure Modes

Finally, it's important to plan ahead to avoid potential failure modes in the first place. Think about what you could have done initially to avoid these failure modes. For example, you could have collected a diverse set of training data to reduce data bias or experimented with different model architectures to find the best solution for your problem.

The key to anticipating failure in your machine learning project is to be proactive rather than reactive. By identifying potential failure modes ahead of time and developing strategies to mitigate them, you'll be better equipped to build a successful machine learning project.

## Task 3: Visualizing ML Results with MF Cards
Now it is time to iterate. Extend the flow in your `baseline_challenge.py` file to include a step that aggregates all of the results from hyperparameter tuning jobs, and logs results and a data visualiation in a Metaflow card.

In [ ]:
%%writefile baseline_challenge.py
# %%writefile baseline_challenge.py
# TODO: In this cell, write your BaselineChallenge flow in the baseline_challenge.py file.

from metaflow import FlowSpec, step, Flow, current, Parameter, IncludeFile, card, current
from metaflow.cards import Table, Markdown, Artifact, Image
import numpy as np 
from dataclasses import dataclass

labeling_function = ... # TODO: Define your labeling function here.

@dataclass
class ModelResult:
    "A custom struct for storing model evaluation results."
    name: None
    params: None
    pathspec: None
    acc: None
    rocauc: None

class BaselineChallenge(FlowSpec):

    split_size = Parameter('split-sz', default=0.2)
    data = IncludeFile('data', default='Womens Clothing E-Commerce Reviews.csv')
    kfold = Parameter('k', default=5)
    scoring = Parameter('scoring', default='accuracy')

    @step
    def start(self):

        import pandas as pd
        import io 
        from sklearn.model_selection import train_test_split
        
        # load dataset packaged with the flow.
        # this technique is convenient when working with small datasets that need to move to remove tasks.
        df = ... 
        # TODO: load the data. 
        # Look up a few lines to the IncludeFile('data', default='Womens Clothing E-Commerce Reviews.csv'). 
        # You can find documentation on IncludeFile here: https://docs.metaflow.org/scaling/data#data-in-local-files


        # filter down to reviews and labels 
        df.columns = ["_".join(name.lower().strip().split()) for name in df.columns]
        df = df[~df.review_text.isna()]
        df['review'] = df['review_text'].astype('str')
        _has_review_df = df[df['review_text'] != 'nan']
        reviews = _has_review_df['review_text']
        labels = _has_review_df.apply(labeling_function, axis=1)
        self.df = pd.DataFrame({'label': labels, **_has_review_df})

        # split the data 80/20, or by using the flow's split-sz CLI argument
        _df = pd.DataFrame({'review': reviews, 'label': labels})
        self.traindf, self.valdf = train_test_split(_df, test_size=self.split_size)
        print(f'num of rows in train set: {self.traindf.shape[0]}')
        print(f'num of rows in validation set: {self.valdf.shape[0]}')

        self.next(self.baseline, self.model)

    @step
    def baseline(self):
        "Compute the baseline"

        from sklearn.metrics import accuracy_score, roc_auc_score
        self._name = "baseline"
        params = "Always predict 1"
        pathspec = f"{current.flow_name}/{current.run_id}/{current.step_name}/{current.task_id}"

        predictions = ... # TODO: predict the majority class
        acc = # TODO: return the accuracy_score of these predictions
         
        rocauc = # TODO: return the roc_auc_score of these predictions
        self.result = ModelResult("Baseline", params, pathspec, acc, rocauc)
        self.next(self.aggregate)

    @step
    def model(self):

        # TODO: import your model if it is defined in another file.

        self._name = "model"
        # NOTE: If you followed the link above to find a custom model implementation, 
            # you will have noticed your model's vocab_sz hyperparameter.
            # Too big of vocab_sz causes an error. Can you explain why? 
        self.hyperparam_set = [{'vocab_sz': 100}, {'vocab_sz': 300}, {'vocab_sz': 500}]  
        pathspec = f"{current.flow_name}/{current.run_id}/{current.step_name}/{current.task_id}"

        self.results = []
        for params in self.hyperparam_set:
            model = .. # TODO: instantiate your custom model here!
            model.fit(X=self.df['review'], y=self.df['label'])
            acc = # TODO: evaluate your custom model in an equivalent way to accuracy_score.
            rocauc = # TODO: evaluate your custom model in an equivalent way to roc_auc_score.
            self.results.append(ModelResult(f"NbowModel - vocab_sz: {params['vocab_sz']}", params, pathspec, acc, rocauc))

        self.next(self.aggregate)

    def add_one(self, rows, result, df):
        "A helper function to load results."
        rows.append([
            Markdown(result.name),
            Artifact(result.params),
            Artifact(result.pathspec),
            Artifact(result.acc),
            Artifact(result.rocauc)
        ])
        df['name'].append(result.name)
        df['accuracy'].append(result.acc)
        return rows, df

    @card # TODO: Set your card type to "corise". 
            # I wonder what other card types there are?
            # https://docs.metaflow.org/metaflow/visualizing-results
            # https://github.com/outerbounds/metaflow-card-altair/blob/main/altairflow.py
    @step
    def aggregate(self, inputs):

        import seaborn as sns
        import matplotlib.pyplot as plt
        from matplotlib import rcParams 
        rcParams.update({'figure.autolayout': True})

        rows = []
        violin_plot_df = {'name': [], 'accuracy': []}
        for task in inputs:
            if task._name == "model": 
                for result in task.results:
                    print(result)
                    rows, violin_plot_df = self.add_one(rows, result, violin_plot_df)
            elif task._name == "baseline":
                print(task.result)
                rows, violin_plot_df = self.add_one(rows, task.result, violin_plot_df)
            else:
                raise ValueError("Unknown task._name type. Cannot parse results.")
            
        current.card.append(Markdown("# All models from this flow run"))

        # TODO: Add a Table of the results to your card! 
        current.card.append(
            Table(
                ..., # TODO: What goes here to populate the Table in the card? 
                headers=["Model name", "Params", "Task pathspec", "Accuracy", "ROCAUC"]
            )
        )
        
        fig, ax = plt.subplots(1,1)
        plt.xticks(rotation=40)
        sns.violinplot(data=violin_plot_df, x="name", y="accuracy", ax=ax)
        
        # TODO: Append the matplotlib fig to the card
        # Docs: https://docs.metaflow.org/metaflow/visualizing-results/easy-custom-reports-with-card-components#showing-plots
        
        self.next(self.end)

    @step
    def end(self):
        pass

if __name__ == '__main__':
    BaselineChallenge()

In [37]:
# Teeny tip in VS code to save time
# To execute the .py files:
# Go to the ‘Folders’ tab, it automatically flags the active file in the directory of interest
# Right click the active file, and you’ll see a ‘Copy path’ option
# Then, python pasted_file_path run  into the command line

! python baseline_challenge.py run --data "/home/workspace/workspaces/full-stack-ml-metaflow-corise-week-2/data/Womens Clothing E-Commerce Reviews.csv"

Metaflow 2.8.6+ob(v1) executing BaselineChallenge for user:sandbox
Validating your flow...
    The graph looks good!
Running pylint...
    baseline_challenge.py:80:22: E0602: Undefined variable 'split_size' (undefined-variable)
    baseline_challenge.py:80:39: E0602: Undefined variable '_test_ratio' (undefined-variable)
    baseline_challenge.py:80:57: E0602: Undefined variable 'split_size' (undefined-variable)
    baseline_challenge.py:80:70: E0602: Undefined variable '_test_ratio' (undefined-variable)
    baseline_challenge.py:80:87: E0602: Undefined variable 'split_size' (undefined-variable)
    baseline_challenge.py:101:39: E0602: Undefined variable '_df' (undefined-variable)
    baseline_challenge.py:104:14: E0602: Undefined variable 'model_test' (undefined-variable)
    Pylint is not happy:
    Fix Pylint warnings listed above or say --no-pylint.

